- https://www.datacamp.com/tutorial/an-introduction-to-using-transformers-and-hugging-face

In [6]:
import pandas as pd
# Load the data from the path
data_path = "../Data/datacamp_workspace_export_2023-11-25 06_40_33.csv"
news_data = pd.read_csv(data_path)




# Show data information
news_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1428 entries, 0 to 1427
Data columns (total 14 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   source_id                        1428 non-null   object 
 1   source_name                      1428 non-null   object 
 2   author                           1317 non-null   object 
 3   title                            1428 non-null   object 
 4   description                      1424 non-null   object 
 5   url                              1427 non-null   object 
 6   url_to_image                     1327 non-null   object 
 7   published_at                     1427 non-null   object 
 8   content                          1262 non-null   object 
 9   top_article                      1426 non-null   float64
 10  engagement_reaction_count        1414 non-null   float64
 11  engagement_comment_count         1414 non-null   float64
 12  engagement_share_cou

In [12]:
news_data.head()

,source_id,source_name,author,title,description,url,url_to_image,published_at,content,top_article,engagement_reaction_count,engagement_comment_count,engagement_share_count,engagement_comment_plugin_count
0,reuters,Reuters,Reuters Editorial,NTSB says Autopilot engaged in 2018 California...,"""The National Transportation Safety Board said...",https://www.reuters.com/article/us-tesla-crash...,https://s4.reutersmedia.net/resources/r/?m=02&...,2019-09-03T16:22:20Z,"""WASHINGTON (Reuters) - The National Transport...",0.0,0.0,0.0,2528.0,0.0
1,the-irish-times,The Irish Times,Eoin Burke-Kennedy,Unemployment falls to post-crash low of 5.2%,Latest monthly figures reflect continued growt...,https://www.irishtimes.com/business/economy/un...,https://www.irishtimes.com/image-creator/?id=1...,2019-09-03T10:32:28Z,"""The States jobless rate fell to 5.2 per cent ...",0.0,6.0,10.0,2.0,0.0
2,the-irish-times,The Irish Times,Deirdre McQuillan,"""Louise Kennedy AW2019: Long coats, sparkling ...",Autumn-winter collection features designer’s g...,https://www.irishtimes.com/\t\t\t\t\t\t\t/life...,https://www.irishtimes.com/image-creator/?id=1...,2019-09-03T14:40:00Z,"""Louise Kennedy is showing off her autumn-wint...",1.0,NaN,NaN,NaN,NaN
3,al-jazeera-english,Al Jazeera English,Al Jazeera,North Korean footballer Han joins Italian gian...,Han is the first North Korean player in the Se...,https://www.aljazeera.com/news/2019/09/north-k...,https://www.aljazeera.com/mritems/Images/2019/...,2019-09-03T17:25:39Z,"""Han Kwang Song, the first North Korean footba...",0.0,0.0,0.0,7.0,0.0
4,bbc-news,BBC News,BBC News,UK government lawyer says proroguing parliamen...,"""The UK government's lawyer, David Johnston ar...",https://www.bbc.co.uk/news/av/uk-scotland-4956...,https://ichef.bbci.co.uk/news/1024/branded_new...,2019-09-03T14:39:21Z,NaN,0.0,0.0,0.0,0.0,0.0


In [21]:
english_texts = news_data[:5]['content']
english_texts

0    "WASHINGTON (Reuters) - The National Transport...
1    "The States jobless rate fell to 5.2 per cent ...
2    "Louise Kennedy is showing off her autumn-wint...
3    "Han Kwang Song, the first North Korean footba...
4                                                  NaN
Name: content, dtype: object

# Language Translation

In [7]:
! pip install transformers sentencepiece
from transformers import MarianTokenizer, MarianMTModel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 1.2 MB/s eta 0:00:00 MB/s eta 0:00:01:010m


In [8]:
# Get the name of the model
model_name = 'Helsinki-NLP/opus-mt-en-fr'

# Get the tokenizer
tokenizer = MarianTokenizer.from_pretrained(model_name)
# Instantiate the model
model = MarianMTModel.from_pretrained(model_name)

/home/vimukthi/miniconda3/envs/pytorch/lib/python3.11/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [25]:
def format_batch_texts(language_code, batch_texts):
    formated_bach = [">>{}<< {}".format(language_code, text) for text in batch_texts]
    
    return formated_bach


def perform_translation(batch_texts, model, tokenizer, language="fr"):

  # Prepare the text data into appropriate format for the model
  formated_batch_texts = format_batch_texts(language, batch_texts)
 
  # Generate translation using model
  translated = model.generate(**tokenizer(formated_batch_texts,

                                          return_tensors="pt", padding=True))

  # Convert the generated tokens indices back into text
  translated_texts = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
 
  return translated_texts

In [26]:
# Check the model translation from the original language (English) to French
translated_texts = perform_translation(english_texts, model, tokenizer)

# Create wrapper to properly format the text
from textwrap import TextWrapper
# Wrap text to 80 characters.
wrapper = TextWrapper(width=80)

for text in translated_texts:
  print("Original text: \n", text)
  print("Translation : \n", text)
  print(print(wrapper.fill(text)))
  print("")

Original text: 
 "WASHINGTON (Reuters) - Le National Transportation Safety Board a déclaré mardi qu'un Tesla Model S était en mode pilote automatique lorsqu'il a heurté un camion d'incendie à Culver City, en Californie, l'une des séries d'écrasements que le conseil enquête sur l'assistance au conducteur de Tesla... [+478 chars]"
Translation : 
 "WASHINGTON (Reuters) - Le National Transportation Safety Board a déclaré mardi qu'un Tesla Model S était en mode pilote automatique lorsqu'il a heurté un camion d'incendie à Culver City, en Californie, l'une des séries d'écrasements que le conseil enquête sur l'assistance au conducteur de Tesla... [+478 chars]"
"WASHINGTON (Reuters) - Le National Transportation Safety Board a déclaré mardi
qu'un Tesla Model S était en mode pilote automatique lorsqu'il a heurté un
camion d'incendie à Culver City, en Californie, l'une des séries d'écrasements
que le conseil enquête sur l'assistance au conducteur de Tesla... [+478 chars]"
None

Original text: 
 "L